### seacells-pyscenic analysis

caution : this analysis needs > 200GB Memory in total. if needed, please split script.

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd 
import os
from SEACells.core import SEACells, summarize_by_SEACell, summarize_by_soft_SEACell
from SEACells.plot import plot_initialization, plot_2D, plot_SEACell_sizes
import loompy as lp
import scanpy.external as sce
# Plotting 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# set a working directory
wdir = "202111-CHIP/scRNA_gca_scenic/3_SEACells/"
os.chdir( wdir )

chr_loom_path = "./CHIP_unfiltered_SCT_normalized.loom"
chr_md_path = "./seurat_md.txt.gz"
chr_anndata_path = "./anndata.h5ad"

In [ ]:
adata = sc.read_loom(chr_loom_path)

In [ ]:
df_from_R = pd.read_csv(chr_md_path, sep = "\t")
df_from_R = df_from_R[["cell","anno_l1" ,"sample", "batch", "response", "time", "CHIP_Binary", "CHIP_Severity", "CHIP_MaxAF"]]
df_from_R.head()

In [ ]:
df_obs = adata.obs[["nGene", "nUMI", "seuratCluster"]]
df_obs = df_obs.reset_index()
df_obs = pd.merge(df_obs, df_from_R, left_on= "CellID", right_on="cell")
df_obs = df_obs.set_index("CellID")
df_obs
adata.obs = df_obs
adata.obs["group"] = adata.obs[["sample", "time"]].apply("-".join, axis = 1)

In [ ]:
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000)

In [ ]:
sc.tl.pca(adata, n_comps = 50, use_highly_variable= True)
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
sce.pp.harmony_integrate(adata, 'group', max_iter_harmony = 10)
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=30, use_rep = "X_pca_harmony")
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.3)

In [ ]:
sc.pl.umap(adata, color=['leiden'],
show=False,
    legend_fontsize=6, frameon=True, title='Leiden')
sc.pl.umap(adata, color=['anno_l1'],
show=False,
    legend_fontsize=6, frameon=True, title='Leiden')

In [ ]:
adata.write(chr_anndata_path)

In [ ]:
df_from_R["group"] = df_from_R[["sample", "time"]].apply("-".join, axis = 1)
list_group = df_from_R["group"].unique().tolist()

In [ ]:
for each_name in list_group:
    each_adata = adata[adata.obs["group"].isin([each_name])]
    # automatically find number of SEACells 
    n_cells = each_adata.obs.shape[0]
    n_SEAcells = int(np.floor(n_cells/75))
    build_kernel_on = 'X_pca_harmony' # to using adjusted PCA
    ## Additional parameters
    n_waypoint_eigs = 10 # Number of eigenvalues to consider when initializing metacells
    print("number of SEACells = ", n_SEAcells)
    raw_ad = sc.AnnData(each_adata.X)
    raw_ad.obs_names, raw_ad.var_names = each_adata.obs_names, each_adata.var_names
    each_adata.raw = raw_ad
    
    model =  SEACells(each_adata, 
                  build_kernel_on=build_kernel_on, 
                  n_SEACells=n_SEAcells, 
                  n_waypoint_eigs=n_waypoint_eigs,
                  convergence_epsilon = 1e-5)
    model.construct_kernel_matrix()
    M = model.kernel_matrix
    sns.clustermap(M.toarray()[:500,:500])
    # Initialize archetypes
    model.initialize_archetypes()
    # Plot the initilization to ensure they are spread across phenotypic space
    plot_initialization(each_adata, model)
    # Check for convergence 
    model.fit(min_iter=10, max_iter=100)
    model.plot_convergence()
    plt.figure(figsize=(3,2))
    sns.distplot((model.A_.T > 0.1).sum(axis=1), kde=False)
    plt.title(f'Non-trivial (> 0.1) assignments per cell')
    plt.xlabel('# Non-trivial SEACell Assignments')
    plt.ylabel('# Cells')
    plt.show()

    plt.figure(figsize=(3,2))
    b = np.partition(model.A_.T, -5)    
    sns.heatmap(np.sort(b[:,-5:])[:, ::-1], cmap='viridis', vmin=0)
    plt.title('Strength of top 5 strongest assignments')
    plt.xlabel('$n^{th}$ strongest assignment')
    plt.show()
    
    SEACell_adata = summarize_by_SEACell(each_adata, SEACells_label='SEACell', summarize_layer='raw')
    SEACell_adata
    SEACell_soft_adata = summarize_by_soft_SEACell(each_adata, model.A_, celltype_label='anno_l1',summarize_layer='raw', minimum_weight=0.05)
    SEACell_soft_adata.obs.head
    plot_2D(each_adata, key='X_umap', colour_metacells=True)
    each_adata.obs['cell'] = each_adata.obs['group'].astype(str) +"-"+ each_adata.obs["cell"]
    each_adata.obs.head
    plot_SEACell_sizes(each_adata, bins=5)
    each_adata.write("./each_anndata/"+each_name+".h5ad")

In [ ]:
fnames = [i+".h5ad" for i in list_group]
list_h5ad = [sc.read_h5ad("/mnt/workspace/202111-CHIP/scRNA_gca_scenic/3_SEACells/each_anndata/"+each_fname) for each_fname in fnames]
adata = list_h5ad[1].concatenate(list_h5ad[2:])
adata.write("./anndata.seacells.h5ad")
adata.obs["Cell"] = adata.obs[["sample", "time", "SEACell"]].apply("-".join, axis = 1)
adata.obs = adata.obs.drop("cell", axis = 1)
adata.obs

In [ ]:
SEACell_adata = summarize_by_SEACell(adata, SEACells_label='Cell', summarize_layer='raw')

In [ ]:
df_meta =( adata.
          obs[["sample", "batch", "response", "time", "CHIP_Binary", "CHIP_Severity", "group","Cell"]].
          drop_duplicates().
          set_index("Cell")
        )

SEACell_adata.obs = pd.merge(SEACell_adata.obs, df_meta, left_index = True, right_index = True, how = "inner")
SEACell_adata.write("./anndata.seacells_summarized.h5ad")

In [ ]:
SEACell_adata = sc.read_h5ad("./anndata.seacells_summarized.h5ad")

In [ ]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(SEACell_adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(SEACell_adata.obs_names) ,
    "nGene": np.array( np.sum(SEACell_adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(SEACell_adata.X.transpose() , axis=0)).flatten() ,
    "Louvain_clusters_Scanpy": np.array( SEACell_adata.obs['leiden'].values ),
    "group": np.array(SEACell_adata.obs['group'].values),
    "time": np.array(SEACell_adata.obs['time'].values),
    "sample": np.array(SEACell_adata.obs['sample'].values),
}
lp.create("SEACells.loom", SEACell_adata.X.transpose(), row_attrs, col_attrs)

### GRN construction based on SEACells


In [ ]:
# set variables for file paths to read from and write to:

# set a working directory
wdir = "/mnt/workspace/202111-CHIP/scRNA_gca_scenic/3_SEACells/"
os.chdir( wdir )

# path to unfiltered loom file (this will be created in the optional steps below)
f_loom_path_unfilt = "SEACells.loom" 

# # path to loom file with basic filtering applied (this will be created in the "initial filtering" step below). Optional.
f_loom_path_scenic = "SEACells.filtered.loom"

# path to anndata object, which will be updated to store Scanpy results as they are generated below
f_anndata_path = "anndata_scenic.h5ad"

# path to pyscenic output
f_pyscenic_output = "pyscenic_output.loom"

# loom output, generated from a combination of Scanpy and pySCENIC results:
#f_final_loom = 'CHIP_sample10p_scenic_integrated-output.loom'
f_final_loom = 'CHIP-scenic-integrated-output.loom'

In [ ]:
adata = sc.read_loom(f_loom_path_unfilt)

In [ ]:
nCells=adata.X.shape[0]

# pySCENIC thresholds
minCountsPerGene=3*.01*nCells # 3 counts in 1% of cells
print("minCountsPerGene: ", minCountsPerGene)

minSamples=.01*nCells # 1% of cells
print("minSamples: ", minSamples)

In [ ]:
# simply compute the number of genes per cell (computers 'n_genes' column)
sc.pp.filter_cells(adata, min_genes=0)
# mito and genes/counts cuts
mito_genes = adata.var_names.str.startswith('MT-')
# for each cell compute fraction of counts in mito genes vs. all genes
adata.obs['percent_mito'] = np.sum(
    adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1).A1

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), dpi=150, sharey=True)

x = adata.obs['n_genes']
x_lowerbound = 1500
x_upperbound = 2000
nbins=100

sns.histplot(x, ax=ax1, bins=nbins, kde = True)
sns.histplot(x, ax=ax2, bins=nbins, kde = True)
sns.histplot(x, ax=ax3, bins=nbins, kde = True)

ax2.set_xlim(0,x_lowerbound)
ax3.set_xlim(x_upperbound, adata.obs['n_genes'].max() )

for ax in (ax1,ax2,ax3): 
  ax.set_xlabel('')

ax1.title.set_text('n_genes')
ax2.title.set_text('n_genes, lower bound')
ax3.title.set_text('n_genes, upper bound')

fig.text(-0.01, 0.5, 'Frequency', ha='center', va='center', rotation='vertical', size='x-large')
fig.text(0.5, 0.0, 'Genes expressed per cell', ha='center', va='center', size='x-large')

fig.tight_layout()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), dpi=150, sharey=True)

x = adata.obs['percent_mito']
x_lowerbound = [0.0, 0.07 ]
x_upperbound = [ 0.10, 0.3 ]
nbins=100

sns.histplot(x, ax=ax1, kde=True, bins=nbins)
sns.histplot(x, ax=ax2, kde=True, bins=int(nbins/(x_lowerbound[1]-x_lowerbound[0])) )
sns.histplot(x, ax=ax3, kde=True, bins=int(nbins/(x_upperbound[1]-x_upperbound[0])) )

ax2.set_xlim(x_lowerbound[0], x_lowerbound[1])
ax3.set_xlim(x_upperbound[0], x_upperbound[1] )
for ax in (ax1,ax2,ax3): 
  ax.set_xlabel('')

ax1.title.set_text('percent_mito')
ax2.title.set_text('percent_mito, lower bound')
ax3.title.set_text('percent_mito, upper bound')

fig.text(-0.01, 0.5, 'Frequency', ha='center', va='center', rotation='vertical', size='x-large')
fig.text(0.5, 0.0, 'Mitochondrial read fraction per cell', ha='center', va='center', size='x-large')

fig.tight_layout()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), dpi=150, sharey=False)

sns.histplot( adata.obs['n_genes'], ax=ax1, kde=True, bins=100)
sns.histplot( adata.obs['n_counts'], ax=ax2, kde=True, bins=100)
sns.histplot( adata.obs['percent_mito'], ax=ax3, kde=True, bins=100)

ax1.title.set_text('Number of genes expressed per cell')
ax2.title.set_text('Counts per cell')
ax3.title.set_text('Mitochondrial read fraction per cell')

fig.text(-0.01, 0.5, 'Frequency', ha='center', va='center', rotation='vertical', size='x-large')

fig.tight_layout()

fig.savefig('filtering_panel_prefilter.pdf', dpi=600, bbox_inches='tight')

In [ ]:
sc.pl.scatter(adata, x='n_counts', y='n_genes', color='percent_mito')

In [ ]:
# not needed : adata = adata[adata.obs['percent_mito'] < 0.15, :]
adata.write( f_anndata_path )

In [ ]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
    "Louvain_clusters_Scanpy": np.array(adata.obs['Louvain_clusters_Scanpy'].values ),
    "group": np.array(adata.obs['group'].values),
    "time": np.array(adata.obs['time'].values),
    "sample": np.array(adata.obs['sample'].values),
}
lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs)

In [ ]:
# save a copy of the raw data
adata.raw = adata

# Total-count normalize (library-size correct) to 10,000 reads/cell
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)

# log transform the data.
sc.pp.log1p(adata)

# identify highly variable genes.
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)

# keep only highly variable genes:
adata = adata[:, adata.var['highly_variable']]

# regress out total counts per cell and the percentage of mitochondrial genes expressed
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'], n_jobs=24)

# scale each gene to unit variance, clip values exceeding SD 10.
sc.pp.scale(adata, max_value=10)

# update the anndata file:
adata.write( f_anndata_path )

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=True)
sce.pp.harmony_integrate(adata, 'group', max_iter_harmony = 20)
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=30, use_rep = "X_pca_harmony")
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution= 0.4)
adata.write( f_anndata_path )

In [ ]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
    "Louvain_clusters_Scanpy": np.array(adata.obs['Louvain_clusters_Scanpy'].values ),
    "group": np.array(adata.obs['group'].values),
    "time": np.array(adata.obs['time'].values),
    "sample": np.array(adata.obs['sample'].values),
}
lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs)

In [ ]:
# transcription factors list, provided from scenic
f_tfs = "/mnt/workspace/202111-CHIP/scRNA_gca_scenic/cisTarget_databases/allTFs_hg38.txt" # human
# f_tfs = "/ddn1/vol1/staging/leuven/stg_00002/lcb/cflerin/resources/allTFs_dmel.txt" # drosophila
# f_tfs = "/ddn1/vol1/staging/leuven/stg_00002/lcb/cflerin/resources/allTFs_mm.txt"   # mouse
# tf_names = load_tf_names( f_tfs )

In [ ]:
!/opt/conda/envs/scenicplus/bin/pyscenic grn {f_loom_path_scenic} {f_tfs} -o adj.csv --num_workers 36

In [ ]:
adjacencies = pd.read_csv("adj.csv", index_col=False, sep=',')
adjacencies[adjacencies["TF"].isin(["NFKB1"])]

In [ ]:
import glob
# ranking databases
f_db_glob = "/mnt/workspace/202111-CHIP/scRNA_gca_scenic/cisTarget_databases/*rankings.feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

# motif databases
f_motif_path = "/mnt/workspace/202111-CHIP/scRNA_gca_scenic/cisTarget_databases/motifs-v9-nr.hgnc-m0.001-o0.0.tbl"

In [ ]:
!/opt/conda/envs/scenicplus/bin/pyscenic ctx adj.csv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 20

In [ ]:
nGenesDetectedPerCellbefore = np.sum(adata.X>0, axis=1)
nGenesDetectedPerCell = pd.Series(nGenesDetectedPerCellbefore)
percentiles = nGenesDetectedPerCell.quantile([0.01, 0.05, 0.10, 0.50, 1])
print(percentiles)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=150)
sns.distplot(nGenesDetectedPerCell, norm_hist=False, kde=False, bins='fd')
for i,x in enumerate(percentiles):
    fig.gca().axvline(x=x, ymin=0,ymax=1, color='red')
    ax.text(x=x, y=ax.get_ylim()[1], s=f'{int(x)} ({percentiles.index.values[i]*100}%)', color='red', rotation=30, size='x-small',rotation_mode='anchor' )
ax.set_xlabel('# of genes')
ax.set_ylabel('# of cells')
fig.tight_layout()

In [ ]:
!/opt/conda/envs/scenicplus/bin/pyscenic aucell \
    {f_loom_path_scenic} \
    reg.csv \
    --auc_threshold 0.01 \
    --output {f_pyscenic_output} \
    --num_workers 20

In [ ]:
import json
import zlib
import base64

# collect SCENIC AUCell output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()

In [ ]:
import umap

# UMAP
runUmap = umap.UMAP(n_neighbors=10, min_dist=0.4, metric='correlation').fit_transform
dr_umap = runUmap( auc_mtx )
pd.DataFrame(dr_umap, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_umap.txt", sep='\t')
# tSNE
tsne = TSNE( n_jobs=20 )
dr_tsne = tsne.fit_transform( auc_mtx )
pd.DataFrame(dr_tsne, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_tsne.txt", sep='\t')

In [ ]:
# scenic output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
meta = json.loads(zlib.decompress(base64.b64decode( lf.attrs.MetaData )))
#exprMat = pd.DataFrame( lf[:,:], index=lf.ra.Gene, columns=lf.ca.CellID)
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
regulons = lf.ra.Regulons
dr_umap = pd.read_csv( 'scenic_umap.txt', sep='\t', header=0, index_col=0 )
dr_tsne = pd.read_csv( 'scenic_tsne.txt', sep='\t', header=0, index_col=0 )
###

In [ ]:
auc_mtx.keys()

In [ ]:
auc_mtx.columns = auc_mtx.columns.str.replace('\(','_(',regex=True)
regulons.dtype.names = tuple( [ x.replace("(","_(") for x in regulons.dtype.names ] )
# regulon thresholds
rt = meta['regulonThresholds']
for i,x in enumerate(rt):
    tmp = x.get('regulon').replace("(","_(")
    x.update( {'regulon': tmp} )

In [ ]:
tsneDF = pd.DataFrame(adata.obsm['X_tsne'], columns=['_X', '_Y'])

Embeddings_X = pd.DataFrame( index=lf.ca.CellID )
Embeddings_X = pd.concat( [
        pd.DataFrame(adata.obsm['X_umap'],index=adata.obs.index)[0] ,
        pd.DataFrame(adata.obsm['X_pca'],index=adata.obs.index)[0] ,
        dr_tsne['X'] ,
        dr_umap['X']
    ], sort=False, axis=1, join='outer' )
Embeddings_X.columns = ['1','2','3','4']

Embeddings_Y = pd.DataFrame( index=lf.ca.CellID )
Embeddings_Y = pd.concat( [
        pd.DataFrame(adata.obsm['X_umap'],index=adata.obs.index)[1] ,
        pd.DataFrame(adata.obsm['X_pca'],index=adata.obs.index)[1] ,
        dr_tsne['Y'] ,
        dr_umap['Y']
    ], sort=False, axis=1, join='outer' )
Embeddings_Y.columns = ['1','2','3','4']

In [ ]:

### metadata
metaJson = {}

metaJson['embeddings'] = [
    {
        "id": -1,
        "name": f"Scanpy t-SNE (highly variable genes)"
    },
    {
        "id": 1,
        "name": f"Scanpy UMAP  (highly variable genes)"
    },
    {
        "id": 2,
        "name": "Scanpy PC1/PC2"
    },
    {
        "id": 3,
        "name": "SCENIC AUC t-SNE"
    },
    {
        "id": 4,
        "name": "SCENIC AUC UMAP"
    },
]

metaJson["clusterings"] = [{
            "id": 0,
            "group": "Scanpy",
            "name": "Scanpy louvain default resolution",
            "clusters": [],
        }]

metaJson["metrics"] = [
        {
            "name": "nUMI"
        }, {
            "name": "nGene"
        }, {
            "name": "Percent_mito"
        }
]

metaJson["annotations"] = [
    {
        "name": "Louvain_clusters_Scanpy",
        "values": list(set( adata.obs['leiden'].astype(str) ))
    },
    #{
    #    "name": "Genotype",
    #    "values": list(set(adata.obs['Genotype'].values))
    #},
    #{
    #    "name": "Timepoint",
    #    "values": list(set(adata.obs['Timepoint'].values))
    #},
    #{
    #    "name": "Sample",
    #    "values": list(set(adata.obs['Sample'].values))
    #}
]

# SCENIC regulon thresholds:
metaJson["regulonThresholds"] = rt

for i in range(max(set([int(x) for x in adata.obs['leiden']])) + 1):
    clustDict = {}
    clustDict['id'] = i
    clustDict['description'] = f'Unannotated Cluster {i + 1}'
    metaJson['clusterings'][0]['clusters'].append(clustDict)
    
clusterings = pd.DataFrame()
clusterings["0"] = adata.obs['leiden'].values.astype(np.int64)

In [ ]:
def dfToNamedMatrix(df):
    arr_ip = [tuple(i) for i in df.values]
    dtyp = np.dtype(list(zip(df.dtypes.index, df.dtypes)))
    arr = np.array(arr_ip, dtype=dtyp)
    return arr

In [ ]:
col_attrs = {
    "CellID": np.array(adata.obs.index),
    "nUMI": np.array(adata.obs['n_counts'].values),
    "nGene": np.array(adata.obs['n_genes'].values),
    "Louvain_clusters_Scanpy": np.array( adata.obs["Louvain_clusters_Scanpy"].values ),
    "group": np.array(adata.obs['group'].values),
    "time": np.array(adata.obs['time'].values),
    "sample": np.array(adata.obs['sample'].values),
    "Percent_mito": np.array(adata.obs['percent_mito'].values),
    "Embedding": dfToNamedMatrix(tsneDF),
    "Embeddings_X": dfToNamedMatrix(Embeddings_X),
    "Embeddings_Y": dfToNamedMatrix(Embeddings_Y),
    "RegulonsAUC": dfToNamedMatrix(auc_mtx)
}

row_attrs = {
    "Gene": lf.ra.Gene,
    "Regulons": regulons,
}

attrs = {
    "title": "sampleTitle",
    "MetaData": json.dumps(metaJson),
    "Genome": 'hg38',
    "SCopeTreeL1": "",
    "SCopeTreeL2": "",
    "SCopeTreeL3": ""
}

# compress the metadata field:
attrs['MetaData'] = base64.b64encode(zlib.compress(json.dumps(metaJson).encode('ascii'))).decode('ascii')

In [ ]:
lp.create(
    filename = f_final_loom ,
    layers=lf[:,:],
    row_attrs=row_attrs, 
    col_attrs=col_attrs, 
    file_attrs=attrs
)
lf.close() # close original pyscenic loom file